# 异步IO编程

CPU的速度远远快于磁盘、网络等IO。在一个线程中，CPU执行代码的速度极快，然而，一旦遇到IO操作，如读写文件、发送网络数据时，就需要等待IO操作完成，才能继续进行下一步操作。这种情况称为同步IO。

在服务器网络编程中，服务器程序通常需要同时处理与多个用户的连接，如果都在一个线程里，一旦线程与一个用户连接进行IO时，其他用户的请求将都被block在这里。

怎么解决呢？

一种方案就是有多线程，或多进程，每个用户的连接都用一个独立的进程去处理，但是系统不能无阴增加线程。由于系统切换线程的开销也很大，所以，一旦线程数量过多，CPU的时间就花在线程切换上了，性能会严重下降。

另一种解决办法就是异步IO：当代码执行到一个耗时的IO操作时，它只发出IO指令，并不等待IO结果，然后就去执行其他代码，一段时间后，当IO返回结果时，再通知CPU进行处理。

```c
do_some_code();
r = async_io_operation();
do_some_code();
if (r.status == finished) {
    do_some_code();
}
```

另一种异步的编程模型是事件驱动式的：

```python
loop = get_event_loop();
while True:
    event = loop.get_event()
    process_event(event) # 异步执行
```

事件驱动在网络编程以及GUI编程中非常常见，比如GUI编程中通过把用户的鼠标、键盘的操作以消息的形式发送到消息队列中，然后GUI程序再从这个消息队列中拿到消息然后处理，但对于各个消息的处理是异步的。

## asyncio

`asyncio`是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。

asyncio的编程模型就是一个消息循环，我们把要异步执行的所有事情都push到这个`Eventloop`中执行，那么当其他任何事情因为IO而异步让出CPU时(`yield`)，都会继续执行这个loop中的其他`Event`。



In [13]:
import asyncio
import threading

@asyncio.coroutine
def hello():
    print('Hello, world (%s)' % threading.currentThread())
    r = yield from asyncio.sleep(5) # 相当于在执行一个IO
    print('Hello, again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()


RuntimeError: This event loop is already running

Hello, world (<_MainThread(MainThread, started 140284852958976)>)
Hello, world (<_MainThread(MainThread, started 140284852958976)>)
Hello, again! (<_MainThread(MainThread, started 140284852958976)>)
Hello, again! (<_MainThread(MainThread, started 140284852958976)>)


下面用`asyncio`的异步连接来获取sina,sohu和163的网站首页

In [20]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s ...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.1\r\nHost: %s\r\n\r\n' %host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.douban.com', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

wget www.163.com ...
wget www.sohu.com ...
wget www.douban.com ...
www.163.com header > HTTP/1.1 200 OK
www.163.com header > Date: Wed, 26 Feb 2020 02:09:59 GMT
www.163.com header > Content-Type: text/html; charset=GBK
www.163.com header > Transfer-Encoding: chunked
www.163.com header > Connection: keep-alive
www.163.com header > Expires: Wed, 26 Feb 2020 02:11:03 GMT
www.163.com header > Server: nginx
www.163.com header > Cache-Control: no-cache,no-store,private
www.163.com header > Age: 16
www.163.com header > Vary: Accept-Encoding
www.163.com header > X-Ser: BC57_dx-lt-yd-shandong-jinan-5-cache-6, BC57_dx-lt-yd-shandong-jinan-5-cache-6, BC57_dx-lt-yd-shandong-jinan-5-cache-6, BC8_dx-guangdong-jiangmen-7-cache-1, BC13_dx-guangdong-jiangmen-7-cache-1
www.163.com header > cdn-user-ip: 119.145.4.82
www.163.com header > cdn-ip: 183.47.233.13
www.163.com header > X-Cache-Remote: HIT
www.163.com header > cdn-source: baishan
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Conten

## async/await

用`asyncio`提供的@`asyncio.coroutine`可以把一个`generator`标记为coroutine类型，然后在coroutine内部用`yield from`调用另一个coroutine实现异步操作。

为了简化并更好地标识异步IO，从Python 3.5开始引入了新的语法`async`和`await`，可以让coroutine的代码更简洁易读

In [ ]:
# 旧的写法
@asyncio.coroutine
def hello():
    print('Hello, world (%s)')
    r = yield from asyncio.sleep(5)
    print('Hello, again! (%s)' )
    
# 新的写法
async def hello():
    print('Hello, world (%s)')
    r = await asyncio.sleep(5)
    print('Hello, again! (%s)' )